In [1]:
import pickle
import pandas as pd

enron_df: pd.DataFrame = pd.read_pickle('enron_students.pkl')

In [2]:
# data cleaning
enron_df['To'] = enron_df['To'].fillna('')
enron_df['From'] = enron_df['From'].fillna('')
enron_df['X-From'] = enron_df['X-From'].fillna('')
enron_df['X-To'] = enron_df['X-To'].fillna('')
enron_df['X-cc'] = enron_df['X-cc'].fillna('')
enron_df['X-bcc'] = enron_df['X-bcc'].fillna('')


In [3]:
# enron_df.to_pickle('enron_students_cleaned.pkl')
import pandas as pd
import preprocessing

# enron_df = pd.read_pickle(r'enron_students_cleaned.pkl')

# preprocessing.data_cleaning(enron_df)

[nltk_data] Downloading package stopwords to C:\Users\Efraim
[nltk_data]     Yosofov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Efraim
[nltk_data]     Yosofov\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


bill info. spoke jeff enroncredit.com ltd. going work since dennis o'connell (london lawyer) responsible group. maybe able clarify jeff's 'positions' hedge backed enroncredit. maybe rod handling jeff's credit. i'd appreciate update. sara \twilliam bradford \t08/29/2000 \t\t \t\t sara \t\t mark paul rod \t\t credit derivative sara, contact either paul radous credit derivative u.s. rod nelson lead credit support enroncredit.com available london, necessary. aware recent trade lunch jeff kinneman thursday discus among thing credit support business. concern offer provide collateral dlj without treasury's approval. bill sara shackleton 08/28/2000 william mark credit derivative seeing credit derivatives. trade originating houston coming jeff kinneman's bond/debt trader booked name investments, inc. ena. can't certain backed enroncredit.com limited. particular, proposed total return bond trade dlj international capital requiring investment post collateral. master party. houston credit looking 

In [4]:
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

def build_lda_model(df, num_topics=3, threshold=0.3):

    # Extract and tokenize the email body
    df['tokens'] = df['email_text'].apply(lambda x: x.split())

    # Create a dictionary representation of the documents
    dictionary = corpora.Dictionary(df['tokens'])
    print("Number of unique tokens before filtering:", len(dictionary))
    dictionary.filter_extremes(no_below=10, no_above=0.3)
    print("Number of unique tokens after filtering:", len(dictionary))

    # Create bag-of-words representation of the documents
    df['bow'] = df['tokens'].apply(dictionary.doc2bow)
    # Filter out documents that result in no words after processing
    bow_corpus = df[df['bow'].map(len) > 0]

    # Check if there are any valid documents left
    if bow_corpus.empty:
        raise ValueError("No valid documents to process. All documents resulted in empty BOW representations.")

    # Build an LDA model
    lda_model = LdaMulticore(corpus=list(bow_corpus['bow']),
                             id2word=dictionary,
                             num_topics=num_topics,
                             chunksize=1000,
                             passes=10,
                             per_word_topics=True,
                             workers=8)

    return lda_model, bow_corpus, dictionary


def get_topic_assignments(lda_model: LdaMulticore, bow_corpus, threshold=0.3):
    
    # Assign documents to topics
    topic_assignments = []
    for bow in bow_corpus:
        topic_probs = lda_model.get_document_topics(bow, minimum_probability=0)
        # Sort topics by probability
        topic_probs = sorted(topic_probs, key=lambda x: x[1], reverse=True)
        # Assign "None of these topics" if the highest probability is below the threshold
        if topic_probs[0][1] < threshold:
            topic_assignments.append("None of these topics")
        else:
            topic_assignments.append(f"Topic {topic_probs[0][0]}")

    return topic_assignments


In [29]:
# get 10 % of the enron rows
enron_df['email_text'] = enron_df['Subject'] + ' ' + enron_df['email_body']

# get all the rows where violated_rules len is > 0
enron_labeled = enron_df[enron_df['violated_rules'].map(len) > 0]

enron_small = enron_df[enron_df['violated_rules'].map(len) == 0].sample(frac=0.4, random_state=42)

# concat enron_small enron_labeled to one df
enron_small = pd.concat([enron_small, enron_labeled])



preprocessing.data_cleaning(enron_small, should_remove_links=True)

removed attachments
parsed contacts
removed links
removed foward text
filled nan values


d:\private\Cyber and Machine learning\Assignments\cyber-ai\preprocessing.py:143: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  enron_df['email_text'].fillna(' ', inplace=True)


removed email headers
removed small words
lemmatized


In [31]:
try:
    lda_model, bow_corpus, dictionary = build_lda_model(enron_small, num_topics=40, threshold=0.3)
    print("Model built successfully with {} topics.".format(lda_model.num_topics))
except ValueError as e:
    print(e)

Number of unique tokens before filtering: 226745
Number of unique tokens after filtering: 86510
Model built successfully with 40 topics.


In [80]:
# save the model, dictionary and bow_corpus
import pickle
import pandas as pd
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore
import os
# save lda model, and dictionary to new folder lda


lda_model.save(os.path.join('lda', 'lda_model'))
dictionary.save(os.path.join('lda', 'dictionary'))

# load the model, dictionary and bow_corpus
# lda_model = LdaMulticore.load('lda_model')
# dictionary = corpora.Dictionary.load('dictionary.pkl')
# bow_corpus = pd.read_pickle('bow_corpus.pkl')
# enron_df['tokens'] = enron_df['email_text'].apply(lambda x: x.split())


# Create bag-of-words representation of the documents
# enron_df['bow'] = enron_df['tokens'].apply(dictionary.doc2bow)
# Filter out documents that result in no words after processing
# bow_corpus = enron_df[enron_df['bow'].map(len) > 0]


In [33]:
def get_most_probable_topic(doc, model: LdaMulticore, get_destribution=False, threshold=0.1):
    if not doc:
        return None  # Handle empty documents
    topic_probs = model.get_document_topics(doc, minimum_probability=threshold)
    if not topic_probs:
        return None  # Handle cases with no significant topic probability
    
    if get_destribution:
        return topic_probs
    
    return max(topic_probs, key=lambda x: x[1])[0]  # Return only the topic index


def get_top_n_probabilities(doc, model, n=5):
    if not doc:
        return None  # Handle empty documents
    topic_probs = model.get_document_topics(doc, minimum_probability=0)
    if not topic_probs:
        return None  # Handle cases with no significant topic probability
    
    return sorted(topic_probs, key=lambda x: x[1], reverse=True)[:n]

# Assume lda_model and valid_df are from your previous output
# bow_corpus['Topic'] = [get_most_probable_topic(doc, lda_model, True) for doc in bow_corpus['bow']]

In [ ]:
pd.set_option('display.max_colwidth', 10000)
bow_corpus[bow_corpus['email_text'].str.contains('ubject')].head(1)

In [34]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# Visualize the topics
vis = gensimvis.prepare(lda_model, bow_corpus['bow'], dictionary)
pyLDAvis.display(vis)

# None topic - 38 35 39 17 33     4        23 21

# legal 29

In [35]:
from gensim.models import CoherenceModel

# Compute Coherence Score
# Calculate perplexity for LDA model
def get_score(model, corpus, dictionary):
    perplexity = model.log_perplexity(list(corpus['bow']))
    print("Perplexity: ", perplexity)

    coherence_model_lda = CoherenceModel(model=model, texts=corpus['tokens'], dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('Coherence Score: ', coherence_lda)


get_score(lda_model, bow_corpus, dictionary)

Perplexity:  -8.62603015769645
Coherence Score:  0.571806007840655


In [10]:
enron_small.columns

Index(['Date', 'From', 'To', 'X-To', 'X-From', 'X-cc', 'X-bcc', 'Subject',
       'email_body', 'verdict', 'violated_rules', 'email_text', 'tokens',
       'bow'],
      dtype='object')

In [37]:
# labeled_1_1 = enron_df[enron_df['violated_rules'].str.contains('2.2')]
# # get the best topic for the labeled_1_1 which is new data
# labeled_1_1['Topic'] = [get_most_probable_topic(doc, lda_model) for doc in labeled_1_1['bow']]


# # labeled_1_1['Topic'] = [get_most_probable_topic(doc, lda_model) for doc in labeled_1_1['bow']]
# labeled_1_1['Topic'].value_counts()
enron_df[enron_df['violated_rules'].str.contains('1.2')].head(1)

,Date,From,To,X-To,X-From,X-cc,X-bcc,Subject,email_body,verdict,violated_rules,email_text,tokens,bow
41,"Tue, 19 Sep 2000 09:35:00 -0700 (PDT)",phillip.allen@enron.com,pallen70@hotmail.com,pallen70@hotmail.com,Phillip K Allen,,,Westgate Proforma-Phillip Allen.xls,---------------------- Forwarded by Phillip K ...,BLOCK,"1.3,2.1,2.3",Westgate Proforma-Phillip Allen.xls ----------...,"[Westgate, Proforma-Phillip, Allen.xls, ------...","[(5, 1), (10, 2), (16, 1), (42, 1), (48, 2), (..."


In [ ]:
labels = ['1.1', '1.3', '1.2', '2.1', '2.2']
label_to_topics = {}
for label in labels:
    print(f"Label {label}")
    labeled = enron_df[enron_df['violated_rules'].str.contains(label)]
    labeled['Topic'] = [get_most_probable_topic(doc, lda_model) for doc in labeled['bow']]
    res = labeled['Topic'].value_counts()
    res = [tuple((x, (y/len(labeled)))) for x, y in res.items() if y/len(labeled) > 0.05]
    print(res)
    print("\n")
    label_to_topics[label] = pd.Series(dict(res))



In [72]:

rule_to_topic_working = {
    'None' : [],
    '1.1': [9, 35, 7, 20], # legal
    '1.2': [1, 9, 7, 18, 20, 35, 24] + [5], # financial data
    '1.3': [1, 35, 9, 7, 39] + [5], # business or financial
    '2.1': [1, 9, 7, 35, 24, 20], # financial data
    '2.2': [1, 9, 18, 7, 17, 16, 24] # business data
}



rule_to_topic = {
    'None' : [38, 35, 39, 17, 33],
    '1.1': [9, 35, 7, 20, 29], # legal
    '1.2': [1, 9, 7, 18, 20, 35, 24] + [5], # financial data
    '1.3': [1, 35, 9, 7, 39] + [5], # business or financial
    '2.1': [1, 9, 7, 35, 24, 20], # financial data
    '2.2': [1, 9, 18, 7, 17, 16, 24] # business data
}


# None topic - 38 35 39 17 33     4        23 21

# legal 29

In [38]:
# none label
labeled_none = enron_df[enron_df['violated_rules'].map(len) == 0]
labeled_none = labeled_none.sample(frac=0.2, random_state=42)
labeled_none['Topic'] = [get_most_probable_topic(doc, lda_model) for doc in labeled_none['bow']]
res = labeled_none['Topic'].value_counts()
res = [tuple((x, (y/len(labeled_none)))) for x, y in res.items() if y/len(labeled_none) > 0.05]
print(res)
print("\n")
label_to_topics['None'] = pd.Series(dict(res))

[(35.0, 0.16503214911995842), (7.0, 0.1535364031954277), (9.0, 0.13877162217748046), (20.0, 0.06691779783940595), (24.0, 0.05895087787664264)]




In [39]:
labels = ['1.1', '1.3', '1.2', '2.1', '2.2', 'None']
# print intersecting and unique topics in label_to_dict_topic
for label1 in labels:
    for label2 in labels:
        if label1 != label2:
            print(f"Intersecting topics between {label1} and {label2}")
            print(label_to_topics[label1].index.intersection(label_to_topics[label2].index))
            print("\n")
            print(f"Unique topics in {label1}")
            print(label_to_topics[label1].index.difference(label_to_topics[label2].index))
            print("\n")
            print(f"Unique topics in {label2}")
            print(label_to_topics[label2].index.difference(label_to_topics[label1].index))
            print("\n")

            

Intersecting topics between 1.1 and 1.3
Index([9.0, 35.0, 7.0], dtype='float64')


Unique topics in 1.1
Index([20.0], dtype='float64')


Unique topics in 1.3
Index([39.0], dtype='float64')


Intersecting topics between 1.1 and 1.2
Index([9.0, 35.0, 7.0, 20.0], dtype='float64')


Unique topics in 1.1
Index([], dtype='float64')


Unique topics in 1.2
Index([18.0, 24.0], dtype='float64')


Intersecting topics between 1.1 and 2.1
Index([9.0, 35.0, 7.0, 20.0], dtype='float64')


Unique topics in 1.1
Index([], dtype='float64')


Unique topics in 2.1
Index([24.0], dtype='float64')


Intersecting topics between 1.1 and 2.2
Index([9.0, 7.0], dtype='float64')


Unique topics in 1.1
Index([20.0, 35.0], dtype='float64')


Unique topics in 2.2
Index([16.0, 17.0, 18.0, 24.0], dtype='float64')


Intersecting topics between 1.1 and None
Index([9.0, 35.0, 7.0, 20.0], dtype='float64')


Unique topics in 1.1
Index([], dtype='float64')


Unique topics in None
Index([24.0], dtype='float64')


Intersecting 

In [78]:
# top_10 = labeled_1_1.head(10)

# top_10['prob'] = [get_top_n_probabilities(doc, lda_model) for doc in top_10['bow']]

def intersecting_list(list1, list2):
    return list(set(list1) & set(list2))

# pd.set_option('display.max_colwidth', 1000)
# top_10.head(5)

def get_label_count_based_on_topic(label):
    labeled_data = enron_df[enron_df['violated_rules'].str.contains(label)].copy()
    labeled_data['Topic'] = [get_top_n_probabilities(doc, lda_model, 3) for doc in labeled_data['bow']]
    # get only the first in each tuple from a list of tuples under 'Topic' column
    labeled_data['Topic'] = labeled_data['Topic'].apply(lambda x: x if x is None else [y[0] for y in x])

    # get the count of each topic comapare with rule_to_topic 1.1
    labeled_data['count'] = labeled_data['Topic'].apply(lambda x:  x if x is None else len(intersecting_list(x, rule_to_topic[label])))
    labeled_data['count_none'] = labeled_data['Topic'].apply(lambda x:  x if x is None else len(intersecting_list(x, rule_to_topic['None'])))
    print(f"Label {label}")
    print(f"length of catching at least 1: {len(labeled_data[labeled_data['count'] > 0 ])}")
    print(f"length of catching at least 2: {len(labeled_data[labeled_data['count'] > 1 ])}")
    print(f"count none at least 1: {len(labeled_data[labeled_data['count_none'] > 0 ])}")
    print(f"count none at least 2: {len(labeled_data[labeled_data['count_none'] > 1 ])}")

    # count none at least 2 and length of count at least 1 and 2
    print(f"length of catching at least 1 and none at least 1: {len(labeled_data[(labeled_data['count'] > 0) & (labeled_data['count_none'] > 0)] )}")
    print(f"length of catching at least 2 and none at least 2: {len(labeled_data[(labeled_data['count'] > 0) & (labeled_data['count_none'] > 1)] )}")
    print(f"length of catching at least 2 and none at least 2: {len(labeled_data[(labeled_data['count'] > 0) & (labeled_data['count_none'] > 2)] )}")


    print(f"out of total of: {len(labeled_data)}")


for label in labels:
    get_label_count_based_on_topic(label)

Label 1.1
length of catching at least 1: 377
length of catching at least 2: 214
count none at least 1: 161
count none at least 2: 16
length of catching at least 1 and none at least 1: 148
length of catching at least 2 and none at least 2: 16
length of catching at least 2 and none at least 2: 0
out of total of: 443


Label 1.3
length of catching at least 1: 27465
length of catching at least 2: 15822
count none at least 1: 15812
count none at least 2: 1513
length of catching at least 1 and none at least 1: 15470
length of catching at least 2 and none at least 2: 1511
length of catching at least 2 and none at least 2: 12
out of total of: 30632
Label 1.2
length of catching at least 1: 1837
length of catching at least 2: 1276
count none at least 1: 690
count none at least 2: 52
length of catching at least 1 and none at least 1: 615
length of catching at least 2 and none at least 2: 44
length of catching at least 2 and none at least 2: 0
out of total of: 1987
Label 2.1
length of catching at least 1: 8064
length of catching at least 2: 4859
count none at least 1: 3099
count none at least 2: 233
length of catching at least 1 and none at least 1: 2718
length of catching at least 2 and none at least 2: 202
length of catching at least 2 and none at least 2: 0
out of total of: 9097
Label 2.2
length of catchin

In [8]:
import preprocessing
import os
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary



def load_lda_model_and_dictionary():
    # Load the model and dictionary
    loaded_model: LdaMulticore = LdaMulticore.load(os.path.join('lda', 'lda_model'))
    loaded_dictionary: corpora.Dictionary = Dictionary.load(os.path.join('lda', 'dictionary'))
    return loaded_model, loaded_dictionary

lda_model, dictionary = load_lda_model_and_dictionary()


def text_to_bow(text, dictionary):
    # Preprocess the text
    preprocessed_text = preprocessing.preprocess_text(text, )
    # Convert the preprocessed text to a bag-of-words using the dictionary
    bow = dictionary.doc2bow(preprocessed_text.split())
    return bow

def predict_topic_destribution(text, lda_model, dictionary):
    # Convert text to the BoW format
    bow_vector = text_to_bow(text, dictionary)
    # Get the topic distribution
    topic_distribution = lda_model.get_document_topics(bow_vector)
    return topic_distribution


def intersecting_list(list1, list2):
    return list(set(list1) & set(list2))

def predict(text, label):
    rule_to_topic = {
    'None' : [38, 35, 39, 17, 33],
    '1.1' : [9, 35, 7, 20, 29], # legal
    '1.2': [1, 9, 7, 18, 20, 35, 24] + [5], # financial data
    '1.3': [1, 35, 9, 7, 39] + [5], # business or financial
    '2.1': [1, 9, 7, 35, 24, 20], # financial data
    '2.2': [1, 9, 18, 7, 17, 16, 24] # business data
    }

    if label not in rule_to_topic:
        return False
    dest = predict_topic_destribution(text, lda_model, dictionary)
    if dest is None:
        return False
    topics = [y[0] for y in dest]
    count = len(intersecting_list(dest, rule_to_topic[label]))
    count_none = len(intersecting_list(dest, rule_to_topic['None']))
    if count > 0 and count_none < 2:
        return True
    else:
        return False

In [11]:
predict("hi my name is jeff", '1.3')

False

In [42]:
labeled_df = enron_df[enron_df['violated_rules'].str.contains('1.1')]


topics = [predict_topic(text, lda_model, dictionary) for text in labeled_df['email_text']]

# Get the most probable topic for each document
most_probable_topics = [max(topic, key=lambda x: x[1])[0] for topic in topics]

# print(most_probable_topics)
# df = pd.DataFrame.from_records(topics, columns =['Topic', 'Probability'])

# df['Topic'].value_counts()

results = {}

for topic in most_probable_topics:
    if topic not in results:
        results[topic] = 1
    else:
        results[topic] += 1

# sort the results by the values
results = dict(sorted(results.items(), key=lambda item: item[1], reverse=True))

# print(results)

# 1.1 - 0, 13, 2, 16, 3, 28, 11, 12, 24, 25

predict_topic('just some text', lda_model, dictionary)

[(0, 0.012500292),
 (1, 0.012500292),
 (2, 0.012500292),
 (3, 0.012500292),
 (4, 0.012500292),
 (5, 0.012500292),
 (6, 0.012500292),
 (7, 0.012500292),
 (8, 0.5124886),
 (9, 0.012500292),
 (10, 0.012500292),
 (11, 0.012500292),
 (12, 0.012500292),
 (13, 0.012500292),
 (14, 0.012500292),
 (15, 0.012500292),
 (16, 0.012500292),
 (17, 0.012500292),
 (18, 0.012500292),
 (19, 0.012500292),
 (20, 0.012500292),
 (21, 0.012500292),
 (22, 0.012500292),
 (23, 0.012500292),
 (24, 0.012500292),
 (25, 0.012500292),
 (26, 0.012500292),
 (27, 0.012500292),
 (28, 0.012500292),
 (29, 0.012500292),
 (30, 0.012500292),
 (31, 0.012500292),
 (32, 0.012500292),
 (33, 0.012500292),
 (34, 0.012500292),
 (35, 0.012500292),
 (36, 0.012500292),
 (37, 0.012500292),
 (38, 0.012500292),
 (39, 0.012500292)]